# Circulation méridienne de Hadley

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

Les fichiers de données au format netcdf (moyennes mensuelles NCEP) doivent être récupérés par FTP (FileZilla) et placés dans le répertoire data :

Vent méridien, vitesse verticale :
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/omega.mon.mean.nc

In [ ]:
%matplotlib inline

import os

import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

from scipy.integrate import cumtrapz

import warnings
warnings.filterwarnings('ignore')

# Traitement des données de vent méridien et de vitesse verticale

In [ ]:
diri="./data/"
fv    = xr.open_dataset(diri+"vwnd.mon.mean.nc")
fw    = xr.open_dataset(diri+"omega.mon.mean.nc")
print(fv)
print(fw)

In [ ]:
year1=input("année de départ : ")
year2=input("année de fin : ")

In [ ]:
fv    = xr.open_dataset(diri+"vwnd.mon.mean.nc").sel(time=slice(year1,year2)).sel(level=slice(1000,100))
fw    = xr.open_dataset(diri+"omega.mon.mean.nc").sel(time=slice(year1,year2)).sel(level=slice(1000,100))

lat  = fv.lat.values
lev = fv.level.values

print(lat)
print(lev)

In [ ]:
seasons=['DJF','JJA','MAM','SON']
months=['January','February','March','April', 'May', 'June', 'July',
        'August', 'September', 'October', 'November', 'December']

# moyenne saisonnière
fv_mean = fv.groupby('time.season').mean('time')
fw_mean = fw.groupby('time.season').mean('time')
v_season = fv_mean['vwnd']
w_season = fw_mean['omega']

# moyenne mensuelle
fv_mean_month = fv.groupby('time.month').mean('time')
v_month = fv_mean_month['vwnd']

# moyenne zonale
vz_season = v_season.mean(axis=3)
wz_season = w_season.mean(axis=3)
vz_month = v_month.mean(axis=3)

# moyenne annuelle
vz_annual=np.mean(vz_season, axis=0)
wz_annual=np.mean(wz_season, axis=0)

print(vz_annual)
print(vz_month)
print(vz_annual)

# Tracés

In [ ]:
levels_wz = np.arange(-0.05,0.052,0.002)
levels_vz =[-7.0, -6.5, -6.0, -5.5, -5.0, -4.5, -4.0, -3.5, -3.0,
 -2.5, -2.0, -1.5, -1.0, -0.5, 0.5, 1.0, 1.5, 2.0, 2.5,
  3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7]

In [ ]:
def plot_zonal_mean(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 100)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-90, 100, 10))
    ax.set_xticks(np.arange(-90, 100, 10)) 
    return ax

In [ ]:
lat1=int(input("latitude sud pour la coupe en moyenne zonale :"))
lat2=int(input("latitude nord pour la coupe en moyenne zonale :"))

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(1, 1, 1)
fig.suptitle('Meridional wind (m/s) and vertical velocity : NCEP '+year1+'-'+year2, fontsize=16)

ax.set_title('Annual mean', fontsize=14)
plot_zonal_mean(ax)
ax.set_xlim(lat1, lat2)
cf = ax.contourf(lat, lev, wz_annual[:,:], levels_wz, cmap='seismic', extend='both')
c = ax.contour(lat, lev, vz_annual[:,:], levels_vz, colors='black', linewidths=1)
plt.clabel(c, levels_vz, fmt='%1.1f')

cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_annual_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(15, 7), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Vertical velocity (Pa/s) and meridional wind (m/s) - zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
	
 ax.set_title(seasons[i], fontsize=14)
 plot_zonal_mean(ax)
 ax.set_xlim(lat1, lat2)
 cf = ax.contourf(lat, lev, wz_season[i,:,:], levels_wz, cmap='seismic', extend='both')
 c = ax.contour(lat, lev, vz_season[i,:,:], levels_vz, colors='black', linewidths=1)
 plt.clabel(c, levels_vz, fmt='%1.1f')

cb = fig.colorbar(cf, ax=axlist[axlist.shape[0]-1], orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('Pa/s', size='small')

figname='./figs/wv_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Diagnostic de la fonction de courant méridienne

Le flux élémentaire de masse vers le nord (unité $kg.s^{-1}$) à travers une ligne de latitude $\phi$ s'écrit :

$$ d\psi = a \cos\phi d\lambda ~ v ~ \frac{dp}{g} $$


$a$ : rayon de la Terre ; $g$ : accélération de la gravité


Ainsi, le flux de masse vers le nord total à travers $\phi$ au-dessus d'un niveau de pression $p$ est donné par :

\begin{align*} 
\psi(p) &= \int_0^{2\pi} \int_0^p a \cos\phi d\lambda ~ v ~ \frac{dp}{g} \\
&= 2\pi a \frac{\cos\phi}{g} \int_0^p [v] dp 
\end{align*}

L'équation de continuité (conservation de la masse) en coordonnée pression et en moyenne zonale s'écrit :

$$ \frac{1}{a \cos\phi} \frac{\partial}{\partial \phi} \left( [\overline{v}] \cos\phi \right) + \frac{\partial [\overline{\omega}]}{\partial p} = 0 $$

Ainsi, le vecteur de composantes $[\overline{v}], [\overline{\omega}]$ est non-divergent. On peut donc l'interpréter par une fonction de courant scalaire $\psi$, la **fonction de courant de transport de masse**:

\begin{align*}
2\pi a \cos\phi [\overline{v}] &= \frac{\partial \psi}{\partial p} \\
2\pi a^2 \cos\phi \left( [\overline{\omega}]\right) = -\frac{\partial \psi}{\partial \phi}
\end{align*}

cohérente avec la définition de $\psi$ ci dessus

On peut donc calculer et tracer la fonction de courant uniquement à partir de données de vent méridien à partir de la formule :

$$ \psi(lev,lat) = \frac{2\pi a \cos\phi}{g} \int_0^p [\overline{v}](lev,lat) dp $$

Les contours de $\psi$ peuvent être interprétés comme des isolignes de la **circulation méridienne moyenne**, c'est-à-dire, la circulation de masse en moyenne zonale et temporelle dans un plan latitude-pression.

<div class="alert alert-danger">
<b>Utiliser la fonction integrate.cumtrapz de scipy ou la méthode cumulative_integrate de xarray pour implémenter le calcul de la fonction de courant méridienne $\psi$.</b>
    
- https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html
- https://docs.scipy.org/doc/scipy-1.4.1/reference/generated/scipy.integrate.cumtrapz.html
    
<br>
    
$\psi(lev,lat)=\frac{2\pi r_e cos(lat)}{g}\int_0^P [v](lev,lat)dP$

</div>

# Tracés

In [ ]:
levels_psi = np.arange(-18,19,1)
levels_psi_an = np.arange(-10,11,1)
levels_psi2 =[-18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7,
 -6, -5, -4, -3, -2, -1, 1, 2, 3, 4,
  5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [ ]:
lat1=int(input("latitude sud pour la coupe en moyenne zonale :"))
lat2=int(input("latitude nord pour la coupe en moyenne zonale :"))

fig = plt.figure(figsize=(15., 8.))
ax = fig.add_subplot(1, 1, 1)
fig.suptitle('Mass meridional streamfunction : NCEP '+year1+'-'+year2, fontsize=16)

ax.set_title('Annual mean', fontsize=14)
plot_zonal_mean(ax)
ax.set_xlim(lat1, lat2)
cf = ax.contourf(lat, lev, psi_annual[:,:], levels_psi_an, cmap='PuOr_r', extend='both')
c = ax.contour(lat, lev, psi_annual[:,:], levels_psi2, colors='black', linewidths=1)
plt.clabel(c, levels_psi2, fmt='%2.1i')

cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
cb.set_label('10$^{10}$ kg/s', size='small')

figname='./figs/psi_zmean_annual_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig, axarr = plt.subplots(nrows=2, ncols=2, figsize=(15, 7), constrained_layout=True)
axlist = axarr.flatten()
fig.suptitle('Mass meridional streamfunction : NCEP '+year1+'-'+year2, fontsize=16)

for i, ax in enumerate(axlist):
    ax.set_title(seasons[i], fontsize=14)
    plot_zonal_mean(ax)
    ax.set_xlim(lat1, lat2)
    cf = ax.contourf(lat, lev, psi_season[i,:,:], levels_psi, cmap='PuOr_r', extend='both')
    c = ax.contour(lat, lev, psi_season[i,:,:], levels_psi2, colors='black', linewidths=1)
    plt.clabel(c, levels_psi2, fmt='%2.1i')

cb = fig.colorbar(cf, orientation='horizontal', shrink=0.74, pad=0)
cb.set_label('10$^{10}$ kg/s', size='small')

figname='./figs/psi_zmean_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
for i in range(12): 
    #print(months[i])
    fig = plt.figure(figsize=(15., 8.))
    fig.suptitle('Mass meridional streamfunction : NCEP '+year1+'-'+year2, fontsize=16)
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(months[i], fontsize=14)
    plot_zonal_mean(ax)
    ax.set_xlim(lat1, lat2)
    
    cf = ax.contourf(lat, lev, psi_month[i,:,:], levels_psi, cmap='PuOr_r', extend='both')
    c = ax.contour(lat, lev, psi_month[i,:,:], levels_psi2, colors='black', linewidths=1)
    plt.clabel(c, levels_psi2, fmt='%2.1i')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05)
    cb.set_label('10$^{10}$ kg/s', size='small')
    
    if i<10:
        figname='./anim/psi_zmean_monclim_0'+str(i)
    if i>=10:
        figname='./anim/psi_zmean_monclim_'+str(i)    
    fig.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
def make_animation():
    nbimages=12
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format('./anim/')))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    cmd = 'rm ./anim/*.png'
    os.system(cmd)
    return Image

In [ ]:
gif_filepath = './anim/psi_zmean_monclim.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)